In [ ]:
"""
training models on the dataset 
before this, we recommend you to fine-tune backbone and warm up gru on the dataset 
"""

import os
from tqdm import tqdm
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import argparse
import random

import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
#from thop import profile

import VideoTransforms as transforms
import arch.backbones as models
from utils.ReadData import Frame_dataset2 as dataset
from utils.FrameOperation import frame_processing
from arch.TemporalAggregation import gru
from prefetch_generator import BackgroundGenerator


class DataLoaderX(torch.utils.data.DataLoader):
    def __iter__(self):
        return BackgroundGenerator(super().__iter__())

model_names = sorted(name for name in models.__dict__
    if name.islower() and not name.startswith("__")
    and callable(models.__dict__[name]))

            
def balanced_loss(action_fits):
    b_loss = 0.0
    action_classes = len(action_fits)
    for i in action_fits:
        b_loss += (i/sum(action_fits) - 1/action_classes) ** 2
    b_l = b_loss#/action_classes
    return b_l
    
def flops_loss(action_fits, flops):
    f_l = sum(action_fits * flops)/sum(action_fits)
    return f_l

    
def main(opts):
    global args, fp
    args = opts
    set_random_seed(args.seed)
    fp = frame_processing(num_station_points = args.num_station_points, 
                          process_type = args.process_type, alpha = args.alpha, modality = args.modality)

    train_dataset = dataset(root = args.data, phase = "train", modality = args.modality)
    val_dataset = dataset(root = args.data, phase = "test", modality = args.modality)
    print("{} samples for training.".format(len(train_dataset)))
    print("{} samples for validation.".format(len(val_dataset)))
  
    train_loader = DataLoaderX(train_dataset,
                               batch_size = args.batch_size,
                               shuffle = True,
                               pin_memory = True,
                               num_workers = args.num_workers,)
    val_loader = DataLoaderX(val_dataset,
                             batch_size = args.batch_size,
                             pin_memory = True,
                             shuffle = False,
                             num_workers = args.num_workers,)

    train_losses = []
    train_acc = []
    val_acc = []

    gflops = torch.tensor([args.gflops["mobilenetv2_0"], args.gflops["mobilenetv2_2"], args.gflops["mobilenetv2_4"],
             args.gflops["mobilenetv2_5"]]).cuda()

    # 如果存在预训练权重则载入
    cnn = models.__dict__[args.arch](pretrained=True, num_classes=args.num_classes).cuda()
    cnn_path = f"./ModelCheckpoints/backbones/{args.arch}_best.pth.tar"
    if os.path.exists(cnn_path):
        par = torch.load(cnn_path, map_location = "cpu")["state_dict"]
        print(f"{args.arch}:", cnn.load_state_dict(par, strict=False))
        cnn.eval()
    else:
        print(f"[WARMING] the pretrained parameter of {args.arch} does not exist!")
    
    net = gru(in_dim = args.in_dim_gru, hidden_dim = args.hidden_dim, num_layers = args.num_layers, dp_ratio = args.dropout, 
                        action_classes = args.action_classes, num_classes = args.num_classes,).cuda() 
    
    if not args.evaluate:
        if args.warmup != "":
            sta = torch.load(args.warmup)
            pretrained_dict = sta['state_dict']
            print(sta["best_prec1"])
            model_dict = net.state_dict()
            pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict 
                               and model_dict[k].numel() == v.numel()}   
            model_dict.update(pretrained_dict)
            print("gru and policy network", net.load_state_dict(model_dict))
    else:
        sta = torch.load(args.pre_dic)
        print(f"train_acc: {sta['train_acc']}, val_acc: {sta['val_acc']}")
        weights_dict = sta['par']
        for k in list(weights_dict.keys()):
            if 'ops' in k or "params" in k:
                del(weights_dict[k])
        load_weights_dict = {k: v for k, v in weights_dict.items() if net.state_dict()[k].numel() == v.numel()}    
        print("gru and policy network", net.load_state_dict(load_weights_dict, strict=False))
        net.eval()
        val_accuracy = validate_one_epoch(epoch = 0, val_loader = val_loader, net = net, cnn = cnn)
        return
        
    #freeze the layer except policy network
    if args.freeze_layers:
        for name, para in net.named_parameters():
            if "policy" not in name:
                para.requires_grad_(False)
            else:
                print("training {}".format(name))
    
    net.init_policy_para()
    pg = [p for p in net.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(pg, lr=args.lr, momentum = 0.9, weight_decay = args.weight_decay)
    loss_function = nn.CrossEntropyLoss().cuda()
    
    best_val_acc = 0.0
    best_train_acc = 0.0
    save_path = r"./ModelCheckpoints/models/checkpoints.pth"
    best_model = r"./ModelCheckpoints/models/model_best.pth"  

    for epoch in range(args.start_epoch, args.epochs):
        '''
        adjust the learning rate according to the epoch
        set the seed for each epoch when training
        anneal the tau down to 0
        '''
        adjust_learning_rate(optimizer = optimizer, epoch = epoch)
        set_random_seed(args.seed + epoch)
        net.policy_tau_decay(epoch)
        
        #train
        net.train()
        train_loss, train_accuracy = train_one_epoch(epoch = epoch, train_loader = train_loader, optimizer = optimizer, loss_function = loss_function,
                    gflops = gflops, net = net, cnn = cnn)
        train_acc.append(train_accuracy)
        train_losses.append(train_loss)
        
        #test
        if (epoch + 1) % args.eval_freq == 0:
            net.eval()
            set_random_seed(args.seed)
            val_accuracy = validate_one_epoch(epoch = epoch, val_loader = val_loader,
                                         net = net, cnn = cnn)
            val_acc.append(val_accuracy)
            state = {'par': net.state_dict(), 'epoch': epoch, "train_acc": train_acc, "val_acc": val_acc, "train_loss": train_loss}
            torch.save(state, save_path)
            if val_accuracy > best_val_acc:
                torch.save(state, best_model)
                best_val_acc = val_accuracy
                if train_accuracy >= best_train_acc:
                    best_train_acc = train_accuracy
            elif val_accuracy == best_val_acc:
                if train_accuracy >= best_train_acc:
                    torch.save(state, best_model)
                    best_train_acc = train_accuracy
    print('Finish Training')
    
    
def train_one_epoch(epoch, train_loader, optimizer, loss_function, 
                    gflops, net, cnn):
    acc = 0.0
    running_loss = 0.0
    train_num = len(train_loader)
    train_bar = tqdm(train_loader, ncols=85)
    
    optimizer.zero_grad()
    for counter, data in enumerate(train_bar):
        step, action, feature, hidden, feature_station_point = 0, 0, 0, None, None
        action_fits = torch.tensor([0]*args.action_classes).cuda()
        images, _, labels = data
        station_point, station_id = fp.get_station_point(frame_list = images)  
        with torch.no_grad():
            feature_station_point = cnn(station_point.cuda(non_blocking = True), extract_feature = True)
        is_keep = True
        while is_keep:
            num = args.action_space[str(action)]
            changed_frame = fp.combine_frame(images[step:step+num], is_training = True, step = step)
            step += num
            next_station_id = next_station_point(station_id, step)
            with torch.no_grad():
                feature = cnn(changed_frame.cuda(non_blocking = True), extract_feature = True)
            action, hidden = net(feature, step, hidden = hidden, 
                            station_point = feature_station_point[next_station_id],
                            train_policy = True)
            action_fits[action] += 1
            is_keep = (step + args.action_space[str(action)]) < len(images)
        changed_frame = fp.combine_frame(images[step:], is_training = True, step = step)
        with torch.no_grad():
            feature = cnn(changed_frame.cuda(non_blocking = True), extract_feature = True)
        train_p = net(feature, step, hidden = hidden, 
                                train_policy = True, check = True)

        train_predict = torch.max(train_p, dim=1)[1]
        acc += torch.eq(train_predict, labels.cuda(non_blocking = True)).sum().item()
        c_l = loss_function(train_p, labels.cuda(non_blocking = True))
        b_l = balanced_loss(action_fits)
        f_l = flops_loss(action_fits, gflops)
        train_loss = c_l + args.balanced_weight[0]*b_l + args.balanced_weight[1]*f_l
        train_loss.requires_grad_(True)
        train_loss.backward()
        
        if (counter + 1) % args.iter_size == 0:
            # compute gradient and do SGD step
            optimizer.step()
            optimizer.zero_grad()
        
        running_loss += train_loss.item()
        train_bar.desc = "train epoch[{}/{}] loss:{:.4f}".format(epoch + 1, args.epochs, train_loss)#训练代码
    
    train_loss = running_loss/train_num
    train_accuracy = acc/train_num
    print('[epoch %d] train_loss: %.4f  train_accuracy: %.4f' %
              (epoch + 1, train_loss, train_accuracy))
    return running_loss, train_accuracy


def validate_one_epoch(epoch, val_loader, net, cnn):
    acc = 0.0
    val_num = len(val_loader)
    with torch.no_grad():
        val_bar = tqdm(val_loader, ncols=85)
        for _, val_data in enumerate(val_bar):
            images, _, labels = val_data
            step, action, feature, hidden, feature_station_point = 0, 0, 0, None, None
            station_point, station_id = fp.get_station_point(frame_list = images)       
            with torch.no_grad():
                feature_station_point = cnn(station_point.cuda(non_blocking = True), extract_feature = True)
            is_keep = True
            while is_keep:
                num = args.action_space[str(action)]
                changed_frame = fp.combine_frame(images[step:step+num], is_training = False, step = step)
                step += num
                next_station_id = next_station_point(station_id, step)
                feature = cnn(changed_frame.cuda(non_blocking = True), extract_feature = True)
                action, hidden = net(feature, step, hidden = hidden, 
                               station_point = feature_station_point[next_station_id],
                               train_policy = True)
                is_keep =  step + args.action_space[str(action)] < len(images)    
            changed_frame = fp.combine_frame(images[step:], is_training = False, step = step)
            feature = cnn(changed_frame.cuda(non_blocking = True), extract_feature = True)
            val_p = net(feature, step, hidden = hidden, 
                              train_policy = True, check = True)
            val_predict = torch.max(val_p, dim= 1)[1]
            acc += torch.eq(val_predict, labels.cuda(non_blocking = True)).sum().item()
            val_bar.desc = "valid epoch[{}/{}]".format(epoch + 1, args.epochs)
    val_accuracy = acc/val_num
    print('[epoch %d] val_accuracy: %.4f' % (epoch + 1, val_accuracy))
    return val_accuracy


def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every x epochs"""
    decay = 0.1 ** (sum(epoch >= np.array(args.lr_steps)))
    lr = args.lr * decay
    print("Current learning rate is %4.6f:" % lr)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

        
def next_station_point(station_point_id, step):
    """select a nearest future station point to the current step"""
    for i in range(len(station_point_id)):
        if station_point_id[i] - step >= 0:
            return i        

def set_random_seed(the_seed):
    """make the training process reproducible"""
    if the_seed >= 0:
        np.random.seed(the_seed)
        torch.manual_seed(the_seed)
        
import argparse
if __name__ == '__main__':
    #是否预训练
    pretrain = True
    parser = argparse.ArgumentParser()
    parser.add_argument('--seed', type = int, default = 1007, help = "the global seed")
    parser.add_argument('--data', metavar='DIR', default='/home/bijianchao/FLAME/',
                    help='path to dataset')
    parser.add_argument('--modality', '-m', metavar='MODALITY', default='rgb',
                    choices=["rgb", "flow"],
                    help='modality: rgb | flow')
    parser.add_argument('--dataset', '-d', default='FLAME',
                    help='dataset: FLAME')
    parser.add_argument('--arch', '-a', metavar='ARCH', default='rgb_mobilenetv2',
                    choices=model_names,)
    parser.add_argument('--new_length', default=1, type=int,
                    metavar='N', help='length of sampled video frames (default: 1)')
    parser.add_argument('--new_width', default=254, type=int,
                    metavar='N', help='resize width (default: 340)')
    parser.add_argument('--new_height', default=254, type=int,
                    metavar='N', help='resize height (default: 256)')
    parser.add_argument('--evaluate', type = bool, default = False, help = "evaluation mode")
    parser.add_argument('--freeze_layers', type = bool, default = True, help = "freeze gru and cnn when training the policy network")
    parser.add_argument('--decay_rate', type = int, default = -0.045, help = "decay rate of tau in the policy network")
    parser.add_argument('--start_epoch', type = int, default = 0)
    parser.add_argument('--epochs', type = int, default = 10)
    parser.add_argument('--eval_freq', type = int, default = 1)
    parser.add_argument('--iter_size', type = int, default = 5)
    parser.add_argument('--num_classes', type = int, default = 2)
    parser.add_argument('--action_classes', type = int, default = 4, help = "action classes of the policy network")
    parser.add_argument('--action_space', type = dict, default = {"0": 1, "1": 3, "2": 5, '3': 7})
    parser.add_argument('--batch_size', type = int, default = 1, help = "batch size of videos")
    parser.add_argument('--in_dim_gru', type = int, default = 1280)
    parser.add_argument('--hidden_dim', type = int, default = 512)
    parser.add_argument('--num_layers', type = int, default = 1, help = "number of gru's layers")
    parser.add_argument('--num_station_points', type = int, default = 2, help = "number of station points")
    parser.add_argument('--num_workers', type = int, default = 4)
    parser.add_argument('--lr', type = float, default = 1.45e-5)
    parser.add_argument('--lr_steps', type = list, default = [25, 40], help = "epoch to adjust learning rate")
    parser.add_argument('--weight_decay', type = float, default = 1e-4)
    parser.add_argument('--balanced_weight', type = float, default = [0.3, 0.1], help = "coefficient to balance losses")
    parser.add_argument('--alpha', type = float, default = 0.3, help = "hyperparameter of beta distribution used in mixup")
    parser.add_argument('--dropout', type = float, default = 0.5)
    parser.add_argument('--gflops', type = dict, default = {'res50_0': 4.12, 'mobilenetv2_0': 0.3, 
                                                            'mobilenetv2_1': 0.27, 'mobilenetv2_2': 0.19, 
                                                            'mobilenetv2_3': 0.13, 'mobilenetv2_4': 0.09, 
                                                            'mobilenetv2_5': 0.04},
                        help = 'gflops of different backbones with different resolution inputs')
    parser.add_argument('--warmup', type = str, default = "./ModelCheckpoints/warmup/rgb_mobilenetv2_gru_best.pth.tar",
                    help = 'weight of well-trained gru')
    parser.add_argument('--pre_dic', type = str, default = "./", help = 'weight of well-trained models for evaluation')
    parser.add_argument('--process_type', type = str, default = 'mixup', help = "methods to combine frames together")
    opt = parser.parse_args(args = [])
    
    if pretrain: 
        main(opt)